In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

from data_preprocessing_utils import data_for_training
from utils import compute_error
from models import get_mlp_model

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tabulate import tabulate

import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.neural_network import MLPRegressor

from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

# We start by loading the pre-processed data

In [ ]:
sales_and_purchase_prices=pd.read_csv('../data/prepocessing/sales_and_purchase_prices.csv')
sales_and_purchase_prices.head()

# we aggregate sales by month

In [ ]:
# Removal of some columns that are not important for prediction: ["InventoryId","Brand","Volume","VendorNo","Amount","VendorName"]
sales_and_purchase_prices=sales_and_purchase_prices.drop(["InventoryId","Brand","Volume","VendorNo","Amount","VendorName","SalesDollars"], axis=1)
sales_and_purchase_prices.head()

In [ ]:

# Convert date columns to datetime format
sales_and_purchase_prices['SalesDate'] = pd.to_datetime(sales_and_purchase_prices['SalesDate'])

# Créez une nouvelle colonne pour l'année et le mois
sales_and_purchase_prices['Year'] = sales_and_purchase_prices['SalesDate'].dt.year
sales_and_purchase_prices['Month'] = sales_and_purchase_prices['SalesDate'].dt.month

# Groupez les données par année et par mois et calculez les agrégats souhaités pour chaque groupe
sales_month_aggregated=sales_and_purchase_prices.groupby(['Year', 'Month','Description','Store','Classification']).agg({
    'SalesQuantity': 'sum',
    'ExciseTax': 'first',
    'Size': 'mean',
    'PurchasePrice': 'first',
    'SalesPrice':'first',
}).reset_index()

sales_month_aggregated.head()

# Deep learning model optimization

In [ ]:
# Assuming sales_month_aggregated is your DataFrame
data = sales_month_aggregated.copy()

#get model input data through the function data_for_training
X_train,X_test,X_val,y_train,y_test,y_val=data_for_training(data)

We're going to use mlp. To start with, we'll create an mlp with minimal parameters to get an idea of performance without model optimization. Then we'll optmize our model using Bayesian optimization.

In [ ]:

best_mlp = MLPRegressor(hidden_layer_sizes=(50), activation='relu', alpha=0.008724528119026307, learning_rate='constant')

# Train the model on the training data
best_mlp.fit(X_train, y_train)

# Make predictions on the validation set with the trained model
y_val_pred = best_mlp.predict(X_val)

# Evaluate the performance of the model on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = best_mlp.score(X_val, y_val)

# Make predictions on the test set with the trained model
y_test_pred = best_mlp.predict(X_test)

# Evaluate the performance of the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = best_mlp.score(X_test, y_test)

print("Validation Set:")
print("MSE (Mean Squared Error) on the validation set:", mse_val)
print("R-squared (Coefficient of Determination) on the validation set:", r2_val)

print("\nTest Set:")
print("MSE (Mean Squared Error) on the test set:", mse_test)
print("R-squared (Coefficient of Determination) on the test set:", r2_test)



Using Bayesian optimization to find the best hyperparameters

In [ ]:

# Objective function for Optuna optimization
def objective(trial):
    # Define the hyperparameters to be optimized
    hidden_layer_one = trial.suggest_categorical('hidden_layer_one', [32, 64, 128, 256,512])
    hidden_layer_two = trial.suggest_categorical('hidden_layer_two', [32, 64, 128, 256,512])
    dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
    dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
    # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128,256,512])
    epochs = trial.suggest_categorical('epochs', [200])



    model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)


    # Define the ModelCheckpoint callback
    checkpoint_filepath = 'model_checkpoint.h5'
    model_checkpoint = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=0
    )

    # Train the model on the training data with validation data and checkpoint callback
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0
                        , validation_data=(X_val, y_val),
                        callbacks=[model_checkpoint])

    # Load the best weights from the saved checkpoint
    model.load_weights(checkpoint_filepath)

    # Evaluate the performance of the model on the test set
    y_test_pred = model.predict(X_test)

    # Calculate performance metrics
    mse_test = mean_squared_error(y_test, y_test_pred)
    # rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

    return mse_test

# Configure Optuna to use the GPU for exhaustive searches
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=1)  # Use n_jobs=1 to avoid parallelism problems on GPU

# Show best hyperparameters found
print("Best hyperparameters:")
print(study.best_params)
print("Best MSE:", study.best_value)


Using Grid Search

In [ ]:

r2=0

for hidden_layer_one in [25,50,75,100,125,150,175,200,225,350,300,350,400]:
  for hidden_layer_two in [325,50,75,100,125,150,175,200,225,350,300,350,400]:
      for dropout_one in [0,0.1,0.2,0.3,0.4,0.5]:
        for dropout_two in [0,0.1,0.2,0.3,0.4,0.5]:
          # for batch_size in [32,64,128,256]:


            model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

            # Define the ModelCheckpoint callback
            checkpoint_filepath = 'model_checkpoint.h5'
            model_checkpoint = ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_best_only=True,
                monitor='val_loss',
                mode='min',
                verbose=0
            )

            # Train the model on the training data with validation data and checkpoint callback
            history = model.fit(X_train, y_train, epochs=300, batch_size=256,
                                verbose=0, validation_data=(X_val, y_val),
                                callbacks=[model_checkpoint])

            # Load the best weights from the saved checkpoint
            model.load_weights(checkpoint_filepath)

            # Evaluate the performance of the model on the test set
            y_test_pred = model.predict(X_test)

            # Calculate performance metrics
            mse_test = mean_squared_error(y_test, y_test_pred)
            rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

            if r2_2>r2:
              r2=r2_2
              print("{},{},{},{}".format(hidden_layer_one,hidden_layer_two,dropout_one,dropout_two) )
              print("RMSE:", rmse2)
              print("Corrélation:", corr2)
              print("MAE:", mae2)
              print("RAE:", rae2)
              print("RRSE:", rrse2)
              print("MAPE:", mape2)
              print("R2:", r2_2)
              print("MSE:", mse_test)
              print("----------------------------------")



Training model with best hyperparamters

In [ ]:
# best parameters

hidden_layer_one=50
hidden_layer_two=100
dropout_one=0
dropout_two=0


model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

# Define the ModelCheckpoint callback
checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint = ModelCheckpoint(
filepath=checkpoint_filepath,
save_best_only=True,
monitor='val_loss',
mode='min',
verbose=0
)

# Train the model on the training data with validation data and checkpoint callback
history = model.fit(X_train, y_train, epochs=300, batch_size=512,
            verbose=2, validation_data=(X_val, y_val),
            callbacks=[model_checkpoint])

# Load the best weights from the saved checkpoint
model.load_weights(checkpoint_filepath)

# Evaluate the performance of the model on the test set
y_test_pred = model.predict(X_test)

# Calculate performance metrics
rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

print("RMSE:", rmse2)
print("Corrélation:", corr2)
print("MAE:", mae2)
print("RAE:", rae2)
print("RRSE:", rrse2)
print("MAPE:", mape2)
print("R2:", r2_2)
print("----------------------------------")

Save the weights of the best model for later loading

In [ ]:
model.save_weights('optimized_model_weights.h5')